<a href="https://colab.research.google.com/github/diogon01/42robotics/blob/master/notebooks/trabalhoV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clonar o repositório do GitHub no ambiente do Colab
!git clone --branch feature/entrega-final https://github.com/felipeabe/artificial-neural-network.git

# Adicionar o diretório src ao sys.path para permitir a importação de módulos
import sys
sys.path.append('/content/artificial-neural-network/src')

# Verificar se o repositório foi clonado corretamente
!ls /content/artificial-neural-network


Cloning into 'artificial-neural-network'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 112 (delta 48), reused 27 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (112/112), 901.24 KiB | 4.98 MiB/s, done.
Resolving deltas: 100% (48/48), done.
datasets  notebooks  README.md	src  teste.ipynb


In [ ]:
# Importando o dataset 'obesity.csv'
# Este conjunto contém dados sobre obesidade com fatores como estilo de vida e transporte
dados_obesidade <- carregar_dataset("/content/artificial-neural-network/datasets/obesity.csv")

# Renomear colunas para nomes mais intuitivos
mapear_colunas <- {
    'family_history_with_overweight': 'historico_familiar',
    'FAVC': 'comida_calorica',
    'FCVC': 'vegetais',
    'NCP': 'refeicoes_dia',
    'SMOKE': 'fuma',
    'CH2O': 'agua_dia',
    'SCC': 'calorias',
    'FAF': 'atividade_fisica',
    'CALC': 'alcool',
    'MTRANS': 'transporte',
    'NObeyesdad': 'nivel_obesidade'
}
dados_obesidade <- renomear_colunas(dados_obesidade, mapear_colunas)

# Remover colunas irrelevantes para a análise
colunas_remover <- ['CAEC', 'TUE']
dados_obesidade <- remover_colunas(dados_obesidade, colunas_remover)

# Codificar variáveis categóricas (binárias e escalares)
mapeamento_binario <- {
    'Gender': {"Female": 0, "Male": 1},
    'historico_familiar': {"yes": 1, "no": 0},
    'comida_calorica': {"yes": 1, "no": 0},
    'fuma': {"yes": 1, "no": 0},
    'calorias': {"yes": 1, "no": 0}
}
dados_obesidade <- codificar_variaveis(dados_obesidade, mapeamento_binario)

# Codificar consumo de álcool e transporte
mapeamento_alcool <- {'no': 0, 'Sometimes': 1, 'Frequently': 2, 'Always': 3}
dados_obesidade <- codificar_escala(dados_obesidade, 'alcool', mapeamento_alcool)

mapeamento_transporte <- {
    'Walking': 0, 'Bike': 0,
    'Motorbike': 1, 'Automobile': 1,
    'Public_Transportation': 1
}
dados_obesidade <- codificar_escala(dados_obesidade, 'transporte', mapeamento_transporte)

# Normalizar dados contínuos (altura, peso, idade)
colunas_normalizar <- ['Height', 'Weight', 'Age']
dados_obesidade <- normalizar_colunas(dados_obesidade, colunas_normalizar)

# Dividir o dataset em treino (80%) e teste (20%)
treino_multiclasse, teste_multiclasse <- dividir_dados(dados_obesidade, proporcao=0.8)


,Gender,Age,Height,Weight,family_overweight,caloric_food,eat_vegetables,meals_day,smoke,water_day,track_calories,physical_activity_week,alcohol,transportation_method,obesity_level
0,0,0.149,0.321,0.187,1,0,2.0,3.0,0,2.0,0,0.0,0,1,1
1,0,0.149,0.132,0.127,1,0,3.0,3.0,1,3.0,1,3.0,1,1,1
2,1,0.191,0.660,0.284,1,0,2.0,3.0,0,2.0,0,2.0,2,1,1
3,1,0.277,0.660,0.358,0,0,3.0,3.0,0,2.0,0,2.0,2,0,2
4,1,0.170,0.623,0.379,0,0,2.0,1.0,0,2.0,0,0.0,1,1,3
5,1,0.319,0.321,0.104,0,1,2.0,3.0,0,2.0,0,0.0,1,1,1
6,0,0.191,0.094,0.119,1,1,3.0,3.0,0,2.0,0,1.0,1,1,1
7,1,0.170,0.358,0.104,0,0,2.0,3.0,0,2.0,0,3.0,1,1,1
8,1,0.213,0.623,0.187,1,1,3.0,3.0,0,2.0,0,1.0,2,1,1
9,1,0.170,0.509,0.216,1,1,2.0,3.0,0,2.0,0,1.0,0,1,1


In [ ]:
# Importar o dataset 'houses.csv'
# Este conjunto contém dados sobre propriedades como preço e área útil
dados_imoveis <- carregar_dataset("/content/artificial-neural-network/datasets/houses.csv")

# Remover colunas irrelevantes
colunas_remover <- ['renovated', 'quartile_zone']
dados_imoveis <- remover_colunas(dados_imoveis, colunas_remover)

# Codificar valores booleanos (True/False)
dados_imoveis <- codificar_booleans(dados_imoveis, {True: 1, False: 0})

# Extrair ano da data de construção
dados_imoveis['ano_construcao'] <- extrair_ano(dados_imoveis['date'])

# Normalizar colunas contínuas (preço e área útil)
colunas_normalizar <- ['price', 'living_in_m2']
dados_imoveis <- normalizar_colunas(dados_imoveis, colunas_normalizar)

# Dividir o dataset em treino (80%) e teste (20%)
treino_regressao, teste_regressao <- dividir_dados(dados_imoveis, proporcao=0.8)


<ipython-input-6-1c407e60d625>:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfr = dfr.replace({True: 1, False: 0})


,date,price,bedrooms,grade,has_basement,living_in_m2,nice_view,perfect_condition,real_bathrooms,has_lavatory,single_floor,month
0,2014,0.215,2,2,1,0.261,0,0,2,1,0,5
1,2014,0.213,2,2,0,0.166,0,0,1,1,0,11
2,2014,0.223,2,2,0,0.201,1,0,1,0,1,12
3,2015,0.171,2,3,0,0.299,0,0,2,1,1,2
4,2015,0.576,3,2,1,0.856,0,0,3,0,0,1
5,2015,0.339,1,2,1,0.103,1,0,1,0,1,3
6,2014,0.149,2,2,1,0.253,0,0,1,1,1,6
7,2014,0.143,1,1,0,0.264,0,0,1,0,1,10
8,2014,0.116,2,2,0,0.166,0,0,1,1,1,7
9,2014,0.432,2,3,0,0.204,0,0,2,1,0,9


In [ ]:
# Importar o dataset 'alzheimer.csv'
# Este conjunto contém informações sobre pacientes e diagnósticos
dados_alzheimer <- carregar_dataset("datasets/alzheimer.csv")

# Remover colunas irrelevantes
colunas_remover <- ['PatientID', 'DoctorInCharge']
dados_alzheimer <- remover_colunas(dados_alzheimer, colunas_remover)

# Separar variáveis categóricas e numéricas
variaveis_categoricas <- ['Gender', 'Ethnicity', 'EducationLevel']
variaveis_numericas <- selecionar_colunas(dados_alzheimer, exceto=variaveis_categoricas + ['Diagnosis'])

# Codificar variáveis categóricas com OneHotEncoder
dados_codificados <- codificar_categoricos(dados_alzheimer, variaveis_categoricas)

# Normalizar variáveis numéricas
dados_numericos <- normalizar_colunas(dados_alzheimer, variaveis_numericas)

# Combinar dados processados
dados_processados <- combinar_dados([dados_codificados, dados_numericos])
dados_processados['Diagnosis'] <- dados_alzheimer['Diagnosis']

# Dividir em treino (80%) e teste (20%)
X_treino, X_teste, y_treino, y_teste <- dividir_dados_com_alvo(dados_processados, alvo='Diagnosis', proporcao=0.8)


Epoch 0/1000, Loss: 0.2501
Epoch 100/1000, Loss: 0.2363
Epoch 200/1000, Loss: 0.2309
Epoch 300/1000, Loss: 0.2278
Epoch 400/1000, Loss: 0.2231
Epoch 500/1000, Loss: 0.2120
Epoch 600/1000, Loss: 0.1911
Epoch 700/1000, Loss: 0.1649
Epoch 800/1000, Loss: 0.1426
Epoch 900/1000, Loss: 0.1280
Acurácia no conjunto de teste utilizando mse: 83.02% 

Epoch 0/1000, Loss: 0.6932
Epoch 100/1000, Loss: 0.6652
Epoch 200/1000, Loss: 0.6531
Epoch 300/1000, Loss: 0.6428
Epoch 400/1000, Loss: 0.6210
Epoch 500/1000, Loss: 0.5708
Epoch 600/1000, Loss: 0.4967
Epoch 700/1000, Loss: 0.4350
Epoch 800/1000, Loss: 0.3992
Epoch 900/1000, Loss: 0.3806
Acurácia no conjunto de teste utilizando cross_entropy: 82.79% 



In [ ]:
# Importar a implementação da Rede Neural
from neural_network import NeuralNetwork
from activations import relu, sigmoid, tanh
from losses import mse_loss, cross_entropy_loss

# Configurar a Rede Neural
configuracao <- {
    "input_size": len(X_treino[0]),
    "hidden_size": 10,
    "activation": "relu",
    "loss": "mse",
    "learning_rate": 0.01,
    "epochs": 1000
}

# Treinar a rede neural com os dados de treino
rede_neural <- NeuralNetwork(configuracao)
rede_neural.treinar(X_treino, y_treino)

# Avaliar a rede neural com os dados de teste
acuracia <- rede_neural.avaliar(X_teste, y_teste)
exibir_resultado("Acurácia do modelo:", acuracia)
